In [1]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import norm

In [2]:
S, K, r, v, q, T, n_reps, n_steps = 41.0, 40.0, 0.08, 0.30, 0.0, 1, 500_000, 252

In [3]:
def asset_paths(S: float, mu: float, sigma: float, div: float, T: int, n_reps: int, n_steps: int) -> np.ndarray:
  dt = T / n_steps
  nudt = (mu - div - 0.5*sigma*sigma)*dt
  sidt = sigma*np.sqrt(dt)
  z = nudt + sidt*np.random.normal(size=(n_reps, n_steps))
  paths = np.cumsum(np.concatenate((np.full((n_reps, 1), np.log(S)), z), axis=1), axis=1)

  return np.exp(paths)

In [4]:
mu = 0.30
paths = asset_paths(S, mu, v, q, T, n_reps, n_steps)

In [5]:
def put_payoff(spot: float, strike: float):
  return np.maximum(strike - spot, 0)

In [6]:
def call_payoff(spot: float, strike: float):
  return np.maximum(spot - strike, 0)

In [7]:
def bsdelta(S, K, r, v, q, T):
  d1 = (np.log(S/K) + (r-q+0.5*v*v)*T) / (v*np.sqrt(T))
  return np.exp(-q*T)*norm.cdf(d1)

In [13]:
def hmc(S, K, mu, r, v, q, T, n_reps, n_steps):
  paths = asset_paths(S, mu, v, q, T, n_reps, n_steps)
  dt = T / n_steps
  df = np.exp(-r*dt)
  cf = put_payoff(paths[:, -1], K)

  for i in reversed(range(1, n_steps - 1)):
    tau = T - i * dt
    h = bsdelta(paths[:, i], K, r, v, q, tau)
    x = paths[:, i]
    s = paths[:, i+1] * df
    p = put_payoff(x, K)
    cf *= df
    cf -= h * (s - x)
    itm = p > 0.0
    #print(i, np.sum(itm))
    fitted = np.polynomial.Polynomial.fit(x[itm], cf[itm], 3)
    c = fitted(x)
    jj = itm & (p > c)
    cf[jj] = p[jj]

  price = df * np.mean(cf)

  return price

In [14]:
prc_hmc = hmc(S, K, mu, r, v, q, T, n_reps, n_steps)
prc_hmc

np.float64(-2.358113057908326)

In [11]:
def lattice(S, K, r, v, q, T, n_steps):
    dt = T/n_steps
    u = np.exp((r - q)*dt + v*np.sqrt(dt))
    d = np.exp((r - q)*dt - v*np.sqrt(dt))
    pu = (np.exp((r - q)*dt) - d) / (u - d)
    pd = 1.0 - pu
    dsc = np.exp(-r * dt)

    ii = np.arange(n_steps + 1)
    x = S * (u ** (n_steps - ii)) * (d ** ii)
    f = put_payoff(x, K)

    for i in range(n_steps - 1, -1, -1):
        for j in range(i + 1):
            f_tmp = dsc * (pu*f[j] + pd*f[j+1])
            x[j] /= u
            f[j] = np.maximum(f_tmp, put_payoff(x[j], K))

    price = np.maximum(f[0], put_payoff(S, K))

    return price

In [15]:
prc_bin = lattice(S, K, r, v, q, T, 500)
prc_bin

np.float64(3.187842568434796)